In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F

'''
from google.colab import drive
drive.mount('/content/drive')
'''

# Set some parameters
BATCH_SIZE = 100 # the higher the better
IMG_WIDTH = 128 # for faster computing on kaggle
IMG_HEIGHT = 128 # for faster computing on kaggle
IMG_CHANNELS = 3

#TRAIN_PATH = '/content/drive/My Drive/UNet/Cell'
#TEST_PATH = '/content/drive/My Drive/UNet/Cell'

TRAIN_PATH = './Cell'
TEST_PATH = './Cell'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42

/home/chenzhong/miniconda3/envs/resnet_anzhe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
#test_ids = next(os.walk(TEST_PATH))[1]
num_test_items = int(0.1 * len(train_ids))

# Slice the train_ids list to get the test IDs
test_ids = train_ids[-num_test_items:]
np.random.seed(10)

In [3]:

import re

#print(len(train_ids))
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.float64)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    
    path = os.path.join(TRAIN_PATH, id_)
    # pattern = r'z\d+'
    # matches = re.finditer(pattern, id_)
    # for match in matches:
    #     start = match.start()
    #     end = match.end()
    #     id_ = id_[:end] + '_mSLIM' + id_[end:]
    img = imread(os.path.join(path, 'images', id_ +'_mSLIM'+ '.png'))
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis=-1).astype(np.float64)
    X_train[n] = img.astype(np.float64)

    # if n == 0:
    #     #print(img.dtype)
    #     plt.figure(figsize=(10, 5))
    #     plt.subplot(1, 2, 1)
    #     plt.imshow(img,cmap='gray')
    #     plt.title("Images")
    #     plt.axis('off')
        
    #     plt.figure(figsize=(10, 5))
    #     plt.subplot(1, 2, 2)
    #     plt.imshow(X_train[n,:,:,0],cmap='gray')
    #X_train[n] = img
    #print(img.shape)
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
    # mask2 = None
    for  mask_file in next(os.walk(path + '/masks/'))[2]:
        mask = imread(os.path.join(path, 'masks', mask_file))
        mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    # mask1_rgb = mask1_color * mask1[:, :, np.newaxis]
    # mask2_rgb = mask2_color * mask2[:, :, np.newaxis]
    # background_rgb = background_color * (1 - (mask1 + mask2)[:, :, np.newaxis])

    Y_train[n] = np.expand_dims(mask,axis=-1)
print('Done!')

Getting and resizing train images and masks ... 


100%|██████████| 808/808 [09:09<00:00,  1.47it/s]

Done!


In [4]:
np.save('X.npy',X_train)
np.save('Y.npy',Y_train)